In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

## Importing custom functions from different folder:

In [2]:
# Get the current directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
folder = 'Implementation'
data_folder = os.path.join(parent_dir, folder)

# Construct the full file path
file_ = 'utils_data'
full_path = os.path.join(data_folder, file_)
print(full_path)
print(data_folder)
sys.path.append(data_folder)  

# Load the functions from .py file  
try:
    from utils_data_multiple import load_multiple_data, plot_multiple_data, point_to_segment, cleaning_clustering_multiple_data
    from utils_data_multiple import plot_data
    from utils_data_multiple import saving_processed_mult_data, load_processed_mult_data, multiple_linear_transf  
    from utils_model_multiple import load_params, fitParamaters_mult
    #from utils_data import plot_velocity
    #from utils_model_multiple import numericalSimulation, ComputeFunctional, ComputeVel
    #from utils_model_multiple import generate_trajectory, plot_simulation, generate_trajectory_vel, optimize_Sigma, plot_multiple_trajectories, plotting_params
except ModuleNotFoundError as e:
    print("ModuleNotFoundError:", e)

/Users/flaviaferrusmarimon/UB/MAM/TFM/Codes1/MotorControlPrediction/Implementation/utils_data
/Users/flaviaferrusmarimon/UB/MAM/TFM/Codes1/MotorControlPrediction/Implementation


## Treating multiple subjects data

Motivation may be: 
- 0: playing alone
- 1: easy rival
- 2: hard rival 

The data is now clustered by motivation level, i.e. `dataTrajecetories-25-M1-C1`. We are also splitting the dataset depending on the playing modes: passing through and stopping. For each dataset moreover we have the four different trajectories.

The data is normalized, clustered in terms of motivation, playing mode and ordered by trajectory. 

We seek to find a paramater fitting for each subject, motivational state and playing mode, and trajectory. Then we can consider the mean for each case to fit the model for each subject and motivational state. 

We consider the mass to be the same at the begining of the movement, hence we are studying 3 motivational states x 2 playing modes parameters per player. 

We have 4 possible trajectories (the playing modes just modify the control function, since the effort is different). 

## Reproducing results

We can reproduce our results in this Notebook by setting the initial parameters as well as on the `main_multiple.py` file. 

Set the following parameters according to the study you wish to conduct: 
- `processing` when set to `True` it conducts the whole processing process from the raw data. This includes: 
    - `cleaning_clustering_multiple_data`: 
        - Cleaning the trajectories which do not cross the target and truncating the ones which do cross it. Also storing the optimal time in which they reach the target, $T^*$, stored in `idxrule` 
        - Classifying the data in 4 clusters, depending on the target they cross
    - `multiple_linear_transf`: 
        - Rotating the needed trajectories and scaling them in order to get normalized data
        - Computing the mean velocity and the velocity profiles for each trajectory

ATTENTION: This process takes more than 2 hours to be fully completed. 

- `fitting` when set to `True` the fitting of the parameters is conducted with the already processed data. This includes: 
    - `fitParamaters_mult`: 
        - Sequential approach: 
            1. Optimal movement duration: Generate the optimal trajectory by optimizing the Functional in terms of the time T 
            2. Optimal controller: generate the optimal trajectory with the optimal time duration by minimizing the functional in terms of the biomechanical parameters $(\alpha, \gamma, \varepsilon)$. 
            3. Stochastic optimization: generate the optimal trajectory with optimal time duration and biomechanical parameters by optimizing the Kolmogorov Sirnov estimate in terms of the sigma 
        - Saving the obtained parameters

- `plotting` when set to `True` all the trajectory plots and prints are explicitly shown on this Notebook
- `saving` when set to `True` all the plots and data processed is saved. 

In [11]:
processing = False
fitting = False
plotting = False
saving = False

### Loading data: 

In [4]:
cluster_points = [
        (10.5, 4.85),   # pt2
        (-11, -5),       # pt3  
        (10.5, 0),      # pt1    
        (-10.5, 1.25)  # pt0
    ]
n_clusters = 4

segments = point_to_segment(cluster_points, n_clusters)

In [15]:
if processing:
    print('Loading and processing data...')
    data_dict = load_multiple_data(first_subj = 25, last_subj = 37, 
                       file_names = 'dataTrajectories' )
    if plotting: 
        print('List with al the target segments:', segments)
        print('List with the two points of segment for cluster 0:', segments[0])
        print('Array with point target 1 for cluster 0:', segments[0][0])
        print('First coordinate of point target 1 for cluster 0:', segments[0][0][0])
        plot_multiple_data(data_dict)
    
    cleaned_data_dict, idxrule_dict = cleaning_clustering_multiple_data(data_dict, 
                                    segments, 
                                    first_subj = 25, last_subj = 37,
                                    save_dir = 'subject_plots')

    if saving: 
        saving_processed_mult_data(idxrule_dict=idxrule_dict, folder_name='clustered_multiple_data')
        saving_processed_mult_data(cleaned_data_dict = cleaned_data_dict, 
                                folder_name = 'clustered_multiple_data')
        
    scaled_data_dict, velocity_dict, results_dict = multiple_linear_transf(cleaned_data_dict, idxrule_dict, 
                            segments, 
                            first_subj = 25, last_subj = 37,
                            n_clusters = 4,
                            saving = True, 
                            save_dir = 'subject_plots_2')
    if saving: 
        saving_processed_mult_data(cleaned_data_dict = scaled_data_dict, 
                                folder_name = 'scaled_data')
        saving_processed_mult_data(cleaned_data_dict = velocity_dict, 
                                folder_name = 'velocity')
        saving_processed_mult_data(idxrule_dict = results_dict, 
                                folder_name = 'scaled_data')
        
    print('Fitting paramaters for the optimized trajectory...')   
    
    if fitting: 
        new_params, opt_sigma = fitParamaters_mult(scaled_data_dict,
                    idxrule_dict, 
                    results_dict,
                    segments, 
                    first_subj = 25, last_subj = 30,
                    n_clusters = 4, folder_name = 'fitted_trajectories_2', 
                    saving = saving)
    
elif fitting: 
    print('Loading processed data...')
    cleaned_data_dict, idxrule_dict = load_processed_mult_data(folder_name='clustered_multiple_data')
    scaled_data_dict, results_dict = load_processed_mult_data(folder_name='scaled_multiple_data') 
    print('Data loaded and processed :)')
    
    print('Fitting paramaters for the optimized trajectory...')   
    new_params, opt_sigma = fitParamaters_mult(scaled_data_dict,
                  idxrule_dict, 
                  results_dict,
                  segments, 
                  first_subj = 25, last_subj = 30,
                  n_clusters = 4, folder_name = 'fitted_trajectories_2', 
                  saving = saving)
    
else: 
    print('Loading fitted parameters...')
    params_loaded = load_params(folder_name = 'fitted_parameters')
    opt_sigma = load_params(folder_name = 'fitted_parameters_sigma')

Loading fitted parameters...
Loading the new parameters...
New parameters have been loaded successfully.
Loading the new parameters...
New parameters have been loaded successfully.


### Fitting the parameters

#### Loading fitted parameters

In [16]:
print(params_loaded.keys())
print(params_loaded[31].keys())
print(params_loaded[31]['31_21_cluster_2'].x)

dict_keys([25, 31, 36, 30, 29, 27, 26, 28, 35, 32, 33, 34])
dict_keys(['31_21_cluster_2', '31_12_cluster_1', '31_32_cluster_3', '31_32_cluster_2', '31_12_cluster_0', '31_21_cluster_3', '31_22_cluster_2', '31_31_cluster_3', '31_11_cluster_1', '31_11_cluster_0', '31_31_cluster_2', '31_22_cluster_3', '31_22_cluster_0', '31_11_cluster_3', '31_31_cluster_1', '31_31_cluster_0', '31_11_cluster_2', '31_22_cluster_1', '31_21_cluster_0', '31_32_cluster_1', '31_12_cluster_3', '31_12_cluster_2', '31_32_cluster_0', '31_21_cluster_1'])
[ 2.94100671  3.03937839 -3.19640915]


In [17]:
print(opt_sigma.keys())
print(opt_sigma[28].keys())
print(opt_sigma[28]['28_32_cluster_3'].x)

dict_keys([25, 31, 36, 30, 29, 27, 26, 28, 35, 32, 33, 34])
dict_keys(['28_32_cluster_3', '28_12_cluster_1', '28_21_cluster_2', '28_21_cluster_3', '28_12_cluster_0', '28_32_cluster_2', '28_11_cluster_1', '28_31_cluster_3', '28_22_cluster_2', '28_22_cluster_3', '28_31_cluster_2', '28_11_cluster_0', '28_31_cluster_1', '28_11_cluster_3', '28_22_cluster_0', '28_22_cluster_1', '28_11_cluster_2', '28_31_cluster_0', '28_12_cluster_3', '28_32_cluster_1', '28_21_cluster_0', '28_21_cluster_1', '28_32_cluster_0', '28_12_cluster_2'])
5.07244087283925
